In [1]:
from pyspark.sql.functions import col, udf, monotonically_increasing_id
from pyspark.sql.types import IntegerType, TimestampType, StringType, BooleanType, StructType, StructField
from pyspark.ml.feature import StringIndexer, VectorAssembler
from pyspark.ml import Pipeline
import plotly.graph_objects as go
import plotly
import datetime

In [2]:
'''importing test data from file storage in Databricks'''
file_location = "/FileStore/tables/test_project.csv"
file_type = "csv"

# CSV options
infer_schema = "true"
first_row_is_header = "true"
delimiter = ","

# The applied options are for CSV files. For other file types, these will be ignored.
testdf = spark.read.format(file_type) \
  .option("inferSchema", infer_schema) \
  .option("header", first_row_is_header) \
  .option("sep", delimiter) \
  .load(file_location)

display(testdf.limit(5))

archived,author,author_cakeday,author_flair_background_color,author_flair_css_class,author_flair_richtext,author_flair_text,author_flair_text_color,author_flair_type,brand_safe,can_gild,contest_mode,created_utc,distinguished,domain,edited,gilded,hidden,hide_score,id,is_crosspostable,is_reddit_media_domain,is_self,is_video,link_flair_css_class,link_flair_richtext,link_flair_text,link_flair_text_color,link_flair_type,locked,media,no_follow,num_comments,num_crossposts,over_18,parent_whitelist_status,permalink,post_hint,preview,retrieved_on,rte_mode,score,secure_media,selftext,send_replies,spoiler,stickied,subreddit,subreddit_id,subreddit_name_prefixed,subreddit_type,suggested_sort,thumbnail,thumbnail_height,thumbnail_width,title,url,whitelist_status
TRUE,spif,null,null,null,[],null,null,text,true,true,false,1143849794,null,democrats.reform.house.gov,false,0,false,false,3rip,true,false,false,false,null,[],null,dark,text,false,null,false,0,0,false,all_ads,/r/reddit.com/comments/3rip/iraq_on_the_record_a_searchable_collection_of_237/,null,null,null,markdown,4,null,null,TRUE,FALSE,FALSE,reddit.com,t5_6,r/reddit.com,archived,null,default,null,null,Iraq on the Record: A Searchable Collection Of 237 Specific Misleading Statements Made By Bush Administration Officials About The Threat Posed By Iraq,http://democrats.reform.house.gov/IraqOnTheRecord/,all_ads
TRUE,jjzak,null,null,null,[],null,null,text,false,true,false,1143849822,null,rubricks.org,false,0,false,false,3rir,true,false,false,false,null,[],null,dark,text,false,null,false,0,0,false,null,/r/ja/comments/3rir/rubricks_cms_on_rails/,null,null,null,markdown,3,null,null,TRUE,FALSE,FALSE,ja,t5_22i6,r/ja,public,new,default,null,null,Rubricks - CMS on Rails,http://rubricks.org/,null
TRUE,toddieg,null,null,null,[],null,null,text,true,true,false,1143849849,null,randomcraponline.com,false,0,false,false,3ris,false,false,false,false,null,[],null,dark,text,false,null,true,1,0,false,all_ads,/r/reddit.com/comments/3ris/islamic_dance_party/,null,null,null,markdown,0,null,null,TRUE,FALSE,FALSE,reddit.com,t5_6,r/reddit.com,archived,null,default,null,null,Islamic Dance Party,http://www.randomcraponline.com/islamic-dance-party/,all_ads
TRUE,rmc,null,null,null,[],null,null,text,true,true,false,1143849910,null,debian-administration.org,false,0,false,false,3riv,true,false,false,false,null,[],null,dark,text,false,null,false,1,0,false,all_ads,/r/reddit.com/comments/3riv/how_to_make_bash_complete_differently_based_on/,null,null,null,markdown,3,null,null,TRUE,FALSE,FALSE,reddit.com,t5_6,r/reddit.com,archived,null,default,null,null,How to make bash complete differently based on the command.,http://www.debian-administration.org/articles/317,all_ads
TRUE,toddieg,null,null,null,[],null,null,text,false,true,false,1143850224,null,randomcraponline.com,false,0,false,false,3rj0,false,false,false,false,null,[],null,dark,text,false,null,true,0,0,true,promo_adult_nsfw,/r/nsfw/comments/3rj0/hot_web_cam_chicks/,null,null,null,markdown,0,null,null,TRUE,FALSE,FALSE,nsfw,t5_vf2,r/nsfw,public,null,default,null,null,Hot Web Cam Chicks,http://www.randomcraponline.com/web-cam-chicks/?bikini-thong/,promo_adult_nsfw


In [3]:
def data_cleaning(df):
  
    df = df.filter(~df.score.contains('""url"":""https:'))
    df = df.filter(df.score.isNotNull())
    print("Number of Sample after dropping Null Score values",df.count())    
    bol_col = ['brand_safe', 'can_gild', 'is_crosspostable', 'no_follow', 'over_18']
    #casting each of these columne into string to change that can be
    for col_name in bol_col:
      df = df.withColumn(col_name, col(col_name).cast("string"))
    
    return df

In [4]:
testData = data_cleaning(testdf)

Number of Sample after dropping Null Score values 12553

In [5]:
bol_col = ['brand_safe', 'can_gild', 'is_crosspostable', 'no_follow', 'over_18']
str_col = ['subreddit_type']
cat_col = bol_col + str_col
int_col = ['num_comments']

In [6]:
%run /Users/shimona.narang@mail.utoronto.ca/feature_engineering

In [7]:
testData = features(testData, cat_col, int_col)

categorized varibales function begin
text transform done
input columns for VA are: ['cat_features', 'w2v', 'num_comments']

In [8]:
testData.columns

Out[35]: ['scaledFeatures', 'score']